# Java CLI

In [46]:
JAVABIN = "java -jar ../javallier/target/scala-2.10/javallier.jar"
PYBIN = "~/py35/bin/pheutil"

print("Check we can access both command line tools:")
!{PYBIN} --version
!{JAVABIN}

Check we can access both command line tools:
pheutil, version 1.0-alpha

usage: javallier COMMAND [OPTIONS]
Javallier CLI - Data61 - 2016
Commands:
    add: Add ENCRYPTED to PLAINTEXT
    genpkey: Create a new paillier keypair
    extract: Extract the public key from a PRIVATE key
    encrypt: Encrypt a value with the given public key
    decrypt: Decrypt CIPHERTEXT with PRIVATEKEY
    multiply: Multiply ENCRYPTED with PLAINTEXT
    addenc: Add ENCRYPTED1 to ENCRYPTED2
Try javallier COMMAND --help for command usage.

Options:

 -h,--help      Show this message and exit.
 -v,--verbose   Enable logging

Please report issues to https://github.com/NICTA/javallier/issues


In [47]:
test_numbers = [0, 1, 1000, 5000, 1e12]

## Test 1 - Encryt with key generated from Python

In [48]:
# Generate the key using Python
!{PYBIN} genpkey --keysize 1024 py_1024_priv.json
!{PYBIN} extract py_1024_priv.json py_1024_public.json
!{JAVABIN} genpkey --keysize 1024 java_1024_priv.json
!{JAVABIN} extract java_1024_priv.json java_1024_public.json

Generating a paillier keypair with keysize of 1024
Keys generated
Private key written to py_1024_priv.json
Loading paillier keypair
Public key written to py_1024_public.json




In [50]:
# Python code encrypt with Java public key, and vice-versa
for i, num in enumerate(test_numbers):
    !{PYBIN} encrypt --output py_{i}.enc java_1024_public.json {num}
    !{JAVABIN} encrypt --output java_{i}.enc py_1024_public.json {num}

Loading public key
Encrypting: +0.0000000000000000

Loading public key
Encrypting: +1.0000000000000000

Loading public key
Encrypting: +1000.0000000000000000

Loading public key
Encrypting: +5000.0000000000000000

Loading public key
Encrypting: +1000000000000.0000000000000000



In [51]:
import json
import base64
from binascii import hexlify

def base64url_decode(payload):
    l = len(payload) % 4
    if l == 2:
        payload += '=='
    elif l == 3:
        payload += '='
    elif l != 0:
        raise ValueError('Invalid base64 string')
    return int(hexlify(base64.urlsafe_b64decode(payload.encode('utf-8'))), 16)

with open('py_1024_public.json') as f:
    print("Python public key encoding:")
    file = json.load(f)
    print(file)
    s = file['n']
    print(s)
    print(base64url_decode(s))
    
    
print("\nJava public key encoding:")
with open('java_1024_public.json') as f:
    file = json.load(f)
    print(file)
    s = file['n']
    print(s)
    print(base64url_decode(s))

Python public key encoding:
{'n': 'wkzSKlLbZ9SDaStZpK_h0a_m2102wsw7sw34TMEck0fLkP6xY2R4OotjP1Dwg2j01ZslblVg5_v8u9r7WRfiXbtbZC4buSfXRbNsYAgd2J4DALxRyvYMRihxd1gCestizytkIDC4o8nBxv_Es0Ka5t2S1mfQKjzU4UViBgehq_E', 'kid': 'Paillier public key generated by pheutil on 2017-02-03 11:28:06', 'alg': 'PAI-GN1', 'key_ops': ['encrypt'], 'kty': 'DAJ'}
wkzSKlLbZ9SDaStZpK_h0a_m2102wsw7sw34TMEck0fLkP6xY2R4OotjP1Dwg2j01ZslblVg5_v8u9r7WRfiXbtbZC4buSfXRbNsYAgd2J4DALxRyvYMRihxd1gCestizytkIDC4o8nBxv_Es0Ka5t2S1mfQKjzU4UViBgehq_E
136442157530465002432140972881076790804030964058949512496532246152877674295270782476070863037256735679734274962621258474906778130503329831296441534477564929984284233083264736251423292955883214668882389824158298357609370808011212064157828495714176776921641072109604088784956997150448492092637235121385776458737

Java public key encoding:
{'n': 'AOjs_oL1IAKCoDj_ZhZnZhHqAxIn0bs89JOjRF8hO3tsm7DQDOSPu06yyT-TOUWK_5uy7S2m7n_qRY5K8w6e4S3yvnq8OYhIxCRMOE_GndWNrXvqNPoKz-3LJvZ1is-GBLZOTVrkvI_thOjK_

In [52]:
# Decrypt from java all the values encrypted bt the python code with the java public key
for i, num in enumerate(test_numbers):
    !{JAVABIN} decrypt --output java_{i}.plaintext java_1024_priv.json py_{i}.enc 
    
    out = open("java_{}.plaintext".format(i)).read().strip()
    print(out)
    print(float(num))


Running the decrypt command
[decrypt, java_1024_priv.json, py_0.enc]
0.0
0.0

Running the decrypt command
[decrypt, java_1024_priv.json, py_1.enc]
1.0
1.0

Running the decrypt command
[decrypt, java_1024_priv.json, py_2.enc]
1000.0
1000.0

Running the decrypt command
[decrypt, java_1024_priv.json, py_3.enc]
5000.0
5000.0

Running the decrypt command
[decrypt, java_1024_priv.json, py_4.enc]
1.0E12
1000000000000.0


In [53]:
# Decrypt from python all the values encrypted bt the java code with the python public key
for i, num in enumerate(test_numbers):
    !{PYBIN} decrypt --output py_{i}.plaintext py_1024_priv.json java_{i}.enc 
    
    out = open("py_{}.plaintext".format(i)).read().strip()
    print(out)
    print(float(num))

Loading private key
Decrypting ciphertext
0
0.0
Loading private key
Decrypting ciphertext
1.0
1.0
Loading private key
Decrypting ciphertext
1000.0
1000.0
Loading private key
Decrypting ciphertext
5000.0
5000.0
Loading private key
Decrypting ciphertext
1000000000000.0
1000000000000.0


# IT WORKS!!!!